### попробуем предсказать цену на жилье по датасету House prices, используя PySpark

Данильченко Вадим

In [1]:
# импортируем библиотеки

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

from tqdm import tqdm

In [2]:
spark

In [3]:
# проверим наличие файла в директории
!ls home/vndanilchenko

train.csv


In [4]:
df = spark.read.csv('/user/vndanilchenko/train.csv', header=True, inferSchema=True)
df.show(5)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [5]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [ ]:
+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|YearBuilt|YearRemodAdd|RoofStyle|RoofMatl|Exterior1st|Exterior2nd|MasVnrType|MasVnrArea|ExterQual|ExterCond|Foundation|BsmtQual|BsmtCond|BsmtExposure|BsmtFinType1|BsmtFinSF1|BsmtFinType2|BsmtFinSF2|BsmtUnfSF|TotalBsmtSF|Heating|HeatingQC|CentralAir|Electrical|1stFlrSF|2ndFlrSF|LowQualFinSF|GrLivArea|BsmtFullBath|BsmtHalfBath|FullBath|HalfBath|BedroomAbvGr|KitchenAbvGr|KitchenQual|TotRmsAbvGrd|Functional|Fireplaces|FireplaceQu|GarageType|GarageYrBlt|GarageFinish|GarageCars|GarageArea|GarageQual|GarageCond|PavedDrive|WoodDeckSF|OpenPorchSF|EnclosedPorch|3SsnPorch|ScreenPorch|PoolArea|PoolQC|Fence|MiscFeature|MiscVal|MoSold|YrSold|SaleType|SaleCondition|SalePrice|
+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
|  1|        60|      RL|         65|   8450|  Pave|   NA|     Reg|        Lvl|   AllPub|   Inside|      Gtl|     CollgCr|      Norm|      Norm|    1Fam|    2Story|          7|          5|     2003|        2003|    Gable| CompShg|    VinylSd|    VinylSd|   BrkFace|       196|       Gd|       TA|     PConc|      Gd|      TA|          No|         GLQ|       706|         Unf|         0|      150|        856|   GasA|       Ex|         Y|     SBrkr|     856|     854|           0|     1710|           1|           0|       2|       1|           3|           1|         Gd|           8|       Typ|         0|         NA|    Attchd|       2003|         RFn|         2|       548|        TA|        TA|         Y|         0|         61|            0|        0|          0|       0|    NA|   NA|         NA|      0|     2|  2008|      WD|       Normal|   208500|
|  2|        20|      RL|         80|   9600|  Pave|   NA|     Reg|        Lvl|   AllPub|      FR2|      Gtl|     Veenker|     Feedr|      Norm|    1Fam|    1Story|          6|          8|     1976|        1976|    Gable| CompShg|    MetalSd|    MetalSd|      None|         0|       TA|       TA|    CBlock|      Gd|      TA|          Gd|         ALQ|       978|         Unf|         0|      284|       1262|   GasA|       Ex|         Y|     SBrkr|    1262|       0|           0|     1262|           0|           1|       2|       0|           3|           1|         TA|           6|       Typ|         1|         TA|    Attchd|       1976|         RFn|         2|       460|        TA|        TA|         Y|       298|          0|            0|        0|          0|       0|    NA|   NA|         NA|      0|     5|  2007|      WD|       Normal|   181500|
|  3|        60|      RL|         68|  11250|  Pave|   NA|     IR1|        Lvl|   AllPub|   Inside|      Gtl|     CollgCr|      Norm|      Norm|    1Fam|    2Story|          7|          5|     2001|        2002|    Gable| CompShg|    VinylSd|    VinylSd|   BrkFace|       162|       Gd|       TA|     PConc|      Gd|      TA|          Mn|         GLQ|       486|         Unf|         0|      434|        920|   GasA|       Ex|         Y|     SBrkr|     920|     866|           0|     1786|           1|           0|       2|       1|           3|           1|         Gd|           6|       Typ|         1|         TA|    Attchd|       2001|         RFn|         2|       608|        TA|        TA|         Y|         0|         42|            0|        0|          0|       0|    NA|   NA|         NA|      0|     9|  2008|      WD|       Normal|   223500|
|  4|        70|      RL|         60|   9550|  Pave|   NA|     IR1|        Lvl|   AllPub|   Corner|      Gtl|     Crawfor|      Norm|      Norm|    1Fam|    2Story|          7|          5|     1915|        1970|    Gable| CompShg|    Wd Sdng|    Wd Shng|      None|         0|       TA|       TA|    BrkTil|      TA|      Gd|          No|         ALQ|       216|         Unf|         0|      540|        756|   GasA|       Gd|         Y|     SBrkr|     961|     756|           0|     1717|           1|           0|       1|       0|           3|           1|         Gd|           7|       Typ|         1|         Gd|    Detchd|       1998|         Unf|         3|       642|        TA|        TA|         Y|         0|         35|          272|        0|          0|       0|    NA|   NA|         NA|      0|     2|  2006|      WD|      Abnorml|   140000|
|  5|        60|      RL|         84|  14260|  Pave|   NA|     IR1|        Lvl|   AllPub|      FR2|      Gtl|     NoRidge|      Norm|      Norm|    1Fam|    2Story|          8|          5|     2000|        2000|    Gable| CompShg|    VinylSd|    VinylSd|   BrkFace|       350|       Gd|       TA|     PConc|      Gd|      TA|          Av|         GLQ|       655|         Unf|         0|      490|       1145|   GasA|       Ex|         Y|     SBrkr|    1145|    1053|           0|     2198|           1|           0|       2|       1|           4|           1|         Gd|           9|       Typ|         1|         TA|    Attchd|       2000|         RFn|         3|       836|        TA|        TA|         Y|       192|         84|            0|        0|          0|       0|    NA|   NA|         NA|      0|    12|  2008|      WD|       Normal|   250000|
+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+

In [6]:
# заполним все na значениями

# None
cols_na2none = ['Alley', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'Exterior2nd', 'Fence', 'FireplaceQu', 'GarageCond', 'GarageFinish', 
                'GarageQual', 'GarageType', 'MasVnrType', 'MiscFeature', 'PoolQC', ]

for col_ in cols_na2none:
    df = df.na.fill({col_:"None"})

# 0
cols_na2zero = ['BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea']    

for col_ in cols_na2zero:
    df = df.na.fill({col_:0})
    
# SBrkr
df = df.na.fill({"Electrical": "SBrkr"})

# VinylSd
df = df.na.fill({"Exterior1st": "VinylSd"})

# Typ
df = df.na.fill({"Functional": "Typ"})

# TA
df = df.na.fill({"KitchenQual": "TA"})

# RL
df = df.na.fill({"MSZoning": "RL"})

# WD
df = df.na.fill({"SaleType": "WD"})

# AllPub
df = df.na.fill({"Utilities": "AllPub"})

In [7]:
# проверим какое-нибудь поле
df.select("Alley").describe().show()

+-------+-----+
|summary|Alley|
+-------+-----+
|  count| 1460|
|   mean| null|
| stddev| null|
|    min| Grvl|
|    max| Pave|
+-------+-----+



In [8]:
# сделаем необходимые замены значений
df = df.replace("Grvl", '1', subset=['Alley']).replace("Pave", '2', subset=['Alley']).replace("NA", '0', subset=['Alley'])
df = df.replace("Po", '1', subset=['BsmtCond']).replace("Fa", '2', subset=['BsmtCond']).replace("TA", '3', subset=['BsmtCond']).replace("Gd", '4', subset=['BsmtCond']) \
    .replace("Ex", '5', subset=['BsmtCond']).replace("NA", '0', subset=['BsmtCond'])
df = df.replace("No", '2', subset=['BsmtExposure']).replace("Mn", '2', subset=['BsmtExposure']).replace("Av", '3', subset=['BsmtExposure']).replace("Gd", '4', subset=['BsmtExposure']).replace("NA", '0', subset=['BsmtExposure'])
df = df.replace("Unf", '1', subset=['BsmtFinType1']).replace("LwQ", '2', subset=['BsmtFinType1']).replace("Rec", '3', subset=['BsmtFinType1']).replace("BLQ", '4', subset=['BsmtFinType1']) \
    .replace("ALQ", '5', subset=['BsmtFinType1']).replace("GLQ", '6', subset=['BsmtFinType1']).replace("NA", '0', subset=['BsmtFinType1'])
df = df.replace("Unf", '1', subset=['BsmtFinType2']).replace("LwQ", '2', subset=['BsmtFinType2']).replace("Rec", '3', subset=['BsmtFinType2']).replace("BLQ", '4', subset=['BsmtFinType2']) \
    .replace("ALQ", '5', subset=['BsmtFinType2']).replace("GLQ", '6', subset=['BsmtFinType2']).replace("NA", '0', subset=['BsmtFinType2'])
df = df.replace("Po", '1', subset=['BsmtQual']).replace("Fa", '2', subset=['BsmtQual']).replace("TA", '3', subset=['BsmtQual']).replace("Gd", '4', subset=['BsmtQual']) \
    .replace("Ex", '5', subset=['BsmtQual']).replace("NA", '0', subset=['BsmtQual'])
df = df.replace("N", '1', subset=['CentralAir']).replace("Y", '2', subset=['CentralAir']).replace("NA", '0', subset=['CentralAir'])
df = df.replace("Po", '1', subset=['ExterCond']).replace("Fa", '2', subset=['ExterCond']).replace("TA", '3', subset=['ExterCond']).replace("Gd", '4', subset=['ExterCond']) \
    .replace("Ex", '5', subset=['ExterCond']).replace("NA", '0', subset=['ExterCond'])
df = df.replace("Po", '1', subset=['ExterQual']).replace("Fa", '2', subset=['ExterQual']).replace("TA", '3', subset=['ExterQual']).replace("Gd", '4', subset=['ExterQual']) \
    .replace("Ex", '5', subset=['ExterQual']).replace("NA", '0', subset=['ExterQual'])
df = df.replace("MnWw", '1', subset=['Fence']).replace("GdWo", '2', subset=['Fence']).replace("MnPrv", '3', subset=['Fence']).replace("GdPrv", '4', subset=['Fence']).replace("NA", '0', subset=['Fence'])
df = df.replace("Po", '1', subset=['FireplaceQu']).replace("Fa", '2', subset=['FireplaceQu']).replace("TA", '3', subset=['FireplaceQu']).replace("Gd", '4', subset=['FireplaceQu']) \
    .replace("Ex", '5', subset=['FireplaceQu']).replace("NA", '0', subset=['FireplaceQu'])
df = df.replace("Sal", '1', subset=['Functional']).replace("Sev", '2', subset=['Functional']).replace("Maj2", '3', subset=['Functional']).replace("Maj1", '4', subset=['Functional']) \
    .replace("Mod", '5', subset=['Functional']).replace("Min2", '6', subset=['Functional']).replace("Min1", '7', subset=['Functional']).replace("Typ", '8', subset=['Functional']).replace("NA", '0', subset=['Functional'])
df = df.replace("Po", '1', subset=['GarageCond']).replace("Fa", '2', subset=['GarageCond']).replace("TA", '3', subset=['GarageCond']).replace("Gd", '4', subset=['GarageCond']) \
    .replace("Ex", '5', subset=['GarageCond']).replace("NA", '0', subset=['GarageCond'])
df = df.replace("Po", '1', subset=['GarageQual']).replace("Fa", '2', subset=['GarageQual']).replace("TA", '3', subset=['GarageQual']).replace("Gd", '4', subset=['GarageQual']) \
    .replace("Ex", '5', subset=['GarageQual']).replace("NA", '0', subset=['GarageQual'])
df = df.replace("Unf", '1', subset=['GarageFinish']).replace("RFn", '2', subset=['GarageFinish']).replace("Fin", '3', subset=['GarageFinish']).replace("NA", '0', subset=['GarageFinish'])
df = df.replace("Po", '1', subset=['HeatingQC']).replace("Fa", '2', subset=['HeatingQC']).replace("TA", '3', subset=['HeatingQC']).replace("Gd", '4', subset=['HeatingQC']) \
    .replace("Ex", '5', subset=['HeatingQC']).replace("NA", '0', subset=['HeatingQC'])
df = df.replace("Po", '1', subset=['KitchenQual']).replace("Fa", '2', subset=['KitchenQual']).replace("TA", '3', subset=['KitchenQual']).replace("Gd", '4', subset=['KitchenQual']) \
    .replace("Ex", '5', subset=['KitchenQual']).replace("NA", '0', subset=['KitchenQual'])
df = df.replace("Low", '1', subset=['LandContour']).replace("HLS", '2', subset=['LandContour']).replace("Bnk", '3', subset=['LandContour']).replace("Lvl", '4', subset=['LandContour']).replace("NA", '0', subset=['LandContour'])
df = df.replace("Sev", '1', subset=['LandSlope']).replace("Mod", '2', subset=['LandSlope']).replace("Gtl", '3', subset=['LandSlope']).replace("NA", '0', subset=['LandSlope'])
df = df.replace("IR3", '1', subset=['LotShape']).replace("IR2", '2', subset=['LotShape']).replace("IR1", '3', subset=['LotShape']).replace("Reg", '4', subset=['LotShape']).replace("NA", '0', subset=['LotShape'])
df = df.replace("N", '0', subset=['PavedDrive']).replace("P", '1', subset=['PavedDrive']).replace("Y", '2', subset=['PavedDrive']).replace("NA", '0', subset=['PavedDrive'])
df = df.replace("Fa", '1', subset=['PoolQC']).replace("TA", '2', subset=['PoolQC']).replace("Gd", '3', subset=['PoolQC']).replace("Ex", '4', subset=['PoolQC']).replace("NA", '0', subset=['PoolQC'])
df = df.replace("Grvl", '1', subset=['Street']).replace("Pave", '2', subset=['Street']).replace("NA", '0', subset=['Street'])
df = df.replace("ELO", '1', subset=['Utilities']).replace("NoSeWa", '2', subset=['Utilities']).replace("NoSewr", '3', subset=['Utilities']).replace("AllPub", '4', subset=['Utilities']).replace("NA", '0', subset=['Utilities'])

# проверим какое-нибудь поле
df.select("Alley").describe().show()

+-------+-------------------+
|summary|              Alley|
+-------+-------------------+
|  count|               1460|
|   mean|0.09041095890410959|
| stddev| 0.3721505895356691|
|    min|                  0|
|    max|                  2|
+-------+-------------------+



In [9]:
# присвоим номер сезона новому полю в зависимости от месяца
seasons = {12 : 0, 1 : 0, 2 : 0, 
           3 : 1, 4 : 1, 5 : 1,
           6 : 2, 7 : 2, 8 : 2, 
           9 : 3, 10 : 3, 11 : 3}

def check_seasons(month_):
    return seasons[month_]


udf_seasons = udf(lambda x: check_seasons(x), returnType=StringType())

df = df.withColumn("SeasonSold", udf_seasons(col("MoSold")))

# проверим результат
df.select(['MoSold', 'SeasonSold']).show(5)

+------+----------+
|MoSold|SeasonSold|
+------+----------+
|     2|         0|
|     5|         1|
|     9|         3|
|     2|         0|
|    12|         0|
+------+----------+
only showing top 5 rows



In [10]:
# посчитаем количество лет
df = df.withColumn("YrActualAge", col("YrSold")-col("YearBuilt"))

# проверим результат
df.select(['YrSold', 'YearBuilt', 'YrActualAge']).show(5)

+------+---------+-----------+
|YrSold|YearBuilt|YrActualAge|
+------+---------+-----------+
|  2008|     2003|          5|
|  2007|     1976|         31|
|  2008|     2001|          7|
|  2006|     1915|         91|
|  2008|     2000|          8|
+------+---------+-----------+
only showing top 5 rows



In [11]:
# проведем дополнительные вычисления
df = df.withColumn("TotalSF1", col("TotalBsmtSF")+col("1stFlrSF")+col("2ndFlrSF"))
df = df.withColumn("TotalSF2", col("BsmtFinSF1")+col("BsmtFinSF2")+col("1stFlrSF")+col("2ndFlrSF"))
df = df.withColumn("AllSF", col("GrLivArea")+col("TotalBsmtSF"))
df = df.withColumn("AllFlrsSF", col("1stFlrSF")+col("2ndFlrSF"))
df = df.withColumn("AllPorchSF", col("OpenPorchSF")+col("EnclosedPorch")+col("3SsnPorch")+col("ScreenPorch"))
df = df.withColumn("TotalBath", 2*(col("FullBath")+0.5*col("HalfBath")+col("BsmtFullBath")+0.5*col("BsmtHalfBath")))

# конвертируем в integer
df = df.withColumn("TotalBath", col("TotalBath").cast(IntegerType()))

df = df.withColumn("TotalPorch", col("OpenPorchSF")+col("3SsnPorch")+col("EnclosedPorch")+col("ScreenPorch")+col("WoodDeckSF"))
df = df.withColumn("OverallScore", col("OverallQual")*col("OverallCond"))
df = df.withColumn("GarageScore", col("GarageQual")*col("GarageCond"))
df = df.withColumn("ExterScore", col("ExterQual")*col("ExterCond"))
df = df.withColumn("KitchenScore", col("KitchenAbvGr")*col("KitchenQual"))
df = df.withColumn("FireplaceScore", col("Fireplaces")*col("FireplaceQu"))
df = df.withColumn("GarageScore", col("GarageArea")*col("GarageQual"))
df = df.withColumn("PoolScore", col("PoolArea")*col("PoolQC"))

# применим конструкцию условного оператора
df = df.withColumn("hasPool", when(col("PoolArea")>0,1).otherwise(0))
df = df.withColumn("has2ndFloor", when(col("2ndFlrSF")>0,1).otherwise(0))
df = df.withColumn("hasGarage", when(col("GarageArea")>0,1).otherwise(0))
df = df.withColumn("hasBsmt", when(col("TotalBsmtSF")>0,1).otherwise(0))
df = df.withColumn("hasFireplace", when(col("Fireplaces")>0,1).otherwise(0))

In [32]:
# сохраним в формате parquet
df.write.parquet('/user/vndanilchenko/train_modified.parquet')

In [35]:
# протестируем можем ли мы прочитать записанный файл
df_m = spark.read.load('/user/vndanilchenko/train_modified.parquet', headers=True, format='parquet')
df_m.show(5)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+----------+-----------+---------+-------+--------+--------+------+---------+----------+----------+------------+-----------+----------+---------

In [ ]:
+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+----------+-----------+---------+-------+--------+--------+------+---------+----------+----------+------------+-----------+----------+------------+--------------+---------+-----------+---------+-------+------------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|YearBuilt|YearRemodAdd|RoofStyle|RoofMatl|Exterior1st|Exterior2nd|MasVnrType|MasVnrArea|ExterQual|ExterCond|Foundation|BsmtQual|BsmtCond|BsmtExposure|BsmtFinType1|BsmtFinSF1|BsmtFinType2|BsmtFinSF2|BsmtUnfSF|TotalBsmtSF|Heating|HeatingQC|CentralAir|Electrical|1stFlrSF|2ndFlrSF|LowQualFinSF|GrLivArea|BsmtFullBath|BsmtHalfBath|FullBath|HalfBath|BedroomAbvGr|KitchenAbvGr|KitchenQual|TotRmsAbvGrd|Functional|Fireplaces|FireplaceQu|GarageType|GarageYrBlt|GarageFinish|GarageCars|GarageArea|GarageQual|GarageCond|PavedDrive|WoodDeckSF|OpenPorchSF|EnclosedPorch|3SsnPorch|ScreenPorch|PoolArea|PoolQC|Fence|MiscFeature|MiscVal|MoSold|YrSold|SaleType|SaleCondition|SalePrice|SeasonSold|YrActualAge|TotalBath|hasPool|TotalSF1|TotalSF2| AllSF|AllFlrsSF|AllPorchSF|TotalPorch|OverallScore|GarageScore|ExterScore|KitchenScore|FireplaceScore|PoolScore|has2ndFloor|hasGarage|hasBsmt|hasFireplace|
+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+----------+-----------+---------+-------+--------+--------+------+---------+----------+----------+------------+-----------+----------+------------+--------------+---------+-----------+---------+-------+------------+
|  1|        60|      RL|         65|   8450|     2|    0|       4|          4|        4|   Inside|        3|     CollgCr|      Norm|      Norm|    1Fam|    2Story|          7|          5|     2003|        2003|    Gable| CompShg|    VinylSd|    VinylSd|   BrkFace|       196|        4|        3|     PConc|       4|       3|           2|           6|       706|           1|         0|      150|        856|   GasA|        5|         2|     SBrkr|     856|     854|           0|     1710|           1|           0|       2|       1|           3|           1|          4|           8|         8|         0|          0|    Attchd|       2003|           2|         2|       548|         3|         3|         2|         0|         61|            0|        0|          0|       0|     0|    0|         NA|      0|     2|  2008|      WD|       Normal|   208500|         0|        5.0|        7|      0|  2566.0|  2416.0|2566.0|   1710.0|      61.0|      61.0|        35.0|     1644.0|      12.0|         4.0|           0.0|      0.0|          1|        1|      1|           0|
|  2|        20|      RL|         80|   9600|     2|    0|       4|          4|        4|      FR2|        3|     Veenker|     Feedr|      Norm|    1Fam|    1Story|          6|          8|     1976|        1976|    Gable| CompShg|    MetalSd|    MetalSd|      None|         0|        3|        3|    CBlock|       4|       3|           4|           5|       978|           1|         0|      284|       1262|   GasA|        5|         2|     SBrkr|    1262|       0|           0|     1262|           0|           1|       2|       0|           3|           1|          3|           6|         8|         1|          3|    Attchd|       1976|           2|         2|       460|         3|         3|         2|       298|          0|            0|        0|          0|       0|     0|    0|         NA|      0|     5|  2007|      WD|       Normal|   181500|         1|       31.0|        5|      0|  2524.0|  2240.0|2524.0|   1262.0|       0.0|     298.0|        48.0|     1380.0|       9.0|         3.0|           3.0|      0.0|          0|        1|      1|           1|
|  3|        60|      RL|         68|  11250|     2|    0|       3|          4|        4|   Inside|        3|     CollgCr|      Norm|      Norm|    1Fam|    2Story|          7|          5|     2001|        2002|    Gable| CompShg|    VinylSd|    VinylSd|   BrkFace|       162|        4|        3|     PConc|       4|       3|           2|           6|       486|           1|         0|      434|        920|   GasA|        5|         2|     SBrkr|     920|     866|           0|     1786|           1|           0|       2|       1|           3|           1|          4|           6|         8|         1|          3|    Attchd|       2001|           2|         2|       608|         3|         3|         2|         0|         42|            0|        0|          0|       0|     0|    0|         NA|      0|     9|  2008|      WD|       Normal|   223500|         3|        7.0|        7|      0|  2706.0|  2272.0|2706.0|   1786.0|      42.0|      42.0|        35.0|     1824.0|      12.0|         4.0|           3.0|      0.0|          1|        1|      1|           1|
|  4|        70|      RL|         60|   9550|     2|    0|       3|          4|        4|   Corner|        3|     Crawfor|      Norm|      Norm|    1Fam|    2Story|          7|          5|     1915|        1970|    Gable| CompShg|    Wd Sdng|    Wd Shng|      None|         0|        3|        3|    BrkTil|       3|       4|           2|           5|       216|           1|         0|      540|        756|   GasA|        4|         2|     SBrkr|     961|     756|           0|     1717|           1|           0|       1|       0|           3|           1|          4|           7|         8|         1|          4|    Detchd|       1998|           1|         3|       642|         3|         3|         2|         0|         35|          272|        0|          0|       0|     0|    0|         NA|      0|     2|  2006|      WD|      Abnorml|   140000|         0|       91.0|        4|      0|  2473.0|  1933.0|2473.0|   1717.0|     307.0|     307.0|        35.0|     1926.0|       9.0|         4.0|           4.0|      0.0|          1|        1|      1|           1|
|  5|        60|      RL|         84|  14260|     2|    0|       3|          4|        4|      FR2|        3|     NoRidge|      Norm|      Norm|    1Fam|    2Story|          8|          5|     2000|        2000|    Gable| CompShg|    VinylSd|    VinylSd|   BrkFace|       350|        4|        3|     PConc|       4|       3|           3|           6|       655|           1|         0|      490|       1145|   GasA|        5|         2|     SBrkr|    1145|    1053|           0|     2198|           1|           0|       2|       1|           4|           1|          4|           9|         8|         1|          3|    Attchd|       2000|           2|         3|       836|         3|         3|         2|       192|         84|            0|        0|          0|       0|     0|    0|         NA|      0|    12|  2008|      WD|       Normal|   250000|         0|        8.0|        7|      0|  3343.0|  2853.0|3343.0|   2198.0|      84.0|     276.0|        40.0|     2508.0|      12.0|         4.0|           3.0|      0.0|          1|        1|      1|           1|
+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+----------+-----------+---------+-------+--------+--------+------+---------+----------+----------+------------+-----------+----------+------------+--------------+---------+-----------+---------+-------+------------+

ОПИСАНИЕ ДАННЫХ


MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density
	
LotFrontage: Linear feet of street connected to property

LotArea: Lot size in square feet

Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved
       	
Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access
		
LotShape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular
       
LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to building
       HLS	Hillside - Significant slope from side to side
       Low	Depression
		
Utilities: Type of utilities available
		
       AllPub	All public Utilities (E,G,W,& S)	
       NoSewr	Electricity, Gas, and Water (Septic Tank)
       NoSeWa	Electricity and Gas Only
       ELO	Electricity only	
	
LotConfig: Lot configuration

       Inside	Inside lot
       Corner	Corner lot
       CulDSac	Cul-de-sac
       FR2	Frontage on 2 sides of property
       FR3	Frontage on 3 sides of property
	
LandSlope: Slope of property
		
       Gtl	Gentle slope
       Mod	Moderate Slope	
       Sev	Severe Slope
	
Neighborhood: Physical locations within Ames city limits

       Blmngtn	Bloomington Heights
       Blueste	Bluestem
       BrDale	Briardale
       BrkSide	Brookside
       ClearCr	Clear Creek
       CollgCr	College Creek
       Crawfor	Crawford
       Edwards	Edwards
       Gilbert	Gilbert
       IDOTRR	Iowa DOT and Rail Road
       MeadowV	Meadow Village
       Mitchel	Mitchell
       Names	North Ames
       NoRidge	Northridge
       NPkVill	Northpark Villa
       NridgHt	Northridge Heights
       NWAmes	Northwest Ames
       OldTown	Old Town
       SWISU	South & West of Iowa State University
       Sawyer	Sawyer
       SawyerW	Sawyer West
       Somerst	Somerset
       StoneBr	Stone Brook
       Timber	Timberland
       Veenker	Veenker
			
Condition1: Proximity to various conditions
	
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
Condition2: Proximity to various conditions (if more than one is present)
		
       Artery	Adjacent to arterial street
       Feedr	Adjacent to feeder street	
       Norm	Normal	
       RRNn	Within 200' of North-South Railroad
       RRAn	Adjacent to North-South Railroad
       PosN	Near positive off-site feature--park, greenbelt, etc.
       PosA	Adjacent to postive off-site feature
       RRNe	Within 200' of East-West Railroad
       RRAe	Adjacent to East-West Railroad
	
BldgType: Type of dwelling
		
       1Fam	Single-family Detached	
       2FmCon	Two-family Conversion; originally built as one-family dwelling
       Duplx	Duplex
       TwnhsE	Townhouse End Unit
       TwnhsI	Townhouse Inside Unit
	
HouseStyle: Style of dwelling
	
       1Story	One story
       1.5Fin	One and one-half story: 2nd level finished
       1.5Unf	One and one-half story: 2nd level unfinished
       2Story	Two story
       2.5Fin	Two and one-half story: 2nd level finished
       2.5Unf	Two and one-half story: 2nd level unfinished
       SFoyer	Split Foyer
       SLvl	Split Level
	
OverallQual: Rates the overall material and finish of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor
	
OverallCond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor
		
YearBuilt: Original construction date

YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)

RoofStyle: Type of roof

       Flat	Flat
       Gable	Gable
       Gambrel	Gabrel (Barn)
       Hip	Hip
       Mansard	Mansard
       Shed	Shed
		
RoofMatl: Roof material

       ClyTile	Clay or Tile
       CompShg	Standard (Composite) Shingle
       Membran	Membrane
       Metal	Metal
       Roll	Roll
       Tar&Grv	Gravel & Tar
       WdShake	Wood Shakes
       WdShngl	Wood Shingles
		
Exterior1st: Exterior covering on house

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast	
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
Exterior2nd: Exterior covering on house (if more than one material)

       AsbShng	Asbestos Shingles
       AsphShn	Asphalt Shingles
       BrkComm	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       CemntBd	Cement Board
       HdBoard	Hard Board
       ImStucc	Imitation Stucco
       MetalSd	Metal Siding
       Other	Other
       Plywood	Plywood
       PreCast	PreCast
       Stone	Stone
       Stucco	Stucco
       VinylSd	Vinyl Siding
       Wd Sdng	Wood Siding
       WdShing	Wood Shingles
	
MasVnrType: Masonry veneer type

       BrkCmn	Brick Common
       BrkFace	Brick Face
       CBlock	Cinder Block
       None	None
       Stone	Stone
	
MasVnrArea: Masonry veneer area in square feet

ExterQual: Evaluates the quality of the material on the exterior 
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
ExterCond: Evaluates the present condition of the material on the exterior
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
Foundation: Type of foundation
		
       BrkTil	Brick & Tile
       CBlock	Cinder Block
       PConc	Poured Contrete	
       Slab	Slab
       Stone	Stone
       Wood	Wood
		
BsmtQual: Evaluates the height of the basement

       Ex	Excellent (100+ inches)	
       Gd	Good (90-99 inches)
       TA	Typical (80-89 inches)
       Fa	Fair (70-79 inches)
       Po	Poor (<70 inches
       NA	No Basement
		
BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA	No Basement
	
BsmtExposure: Refers to walkout or garden level walls

       Gd	Good Exposure
       Av	Average Exposure (split levels or foyers typically score average or above)	
       Mn	Mimimum Exposure
       No	No Exposure
       NA	No Basement
	
BsmtFinType1: Rating of basement finished area

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement
		
BsmtFinSF1: Type 1 finished square feet

BsmtFinType2: Rating of basement finished area (if multiple types)

       GLQ	Good Living Quarters
       ALQ	Average Living Quarters
       BLQ	Below Average Living Quarters	
       Rec	Average Rec Room
       LwQ	Low Quality
       Unf	Unfinshed
       NA	No Basement

BsmtFinSF2: Type 2 finished square feet

BsmtUnfSF: Unfinished square feet of basement area

TotalBsmtSF: Total square feet of basement area

Heating: Type of heating
		
       Floor	Floor Furnace
       GasA	Gas forced warm air furnace
       GasW	Gas hot water or steam heat
       Grav	Gravity furnace	
       OthW	Hot water or steam heat other than gas
       Wall	Wall furnace
		
HeatingQC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor
		
CentralAir: Central air conditioning

       N	No
       Y	Yes
		
Electrical: Electrical system

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed
		
1stFlrSF: First Floor square feet
 
2ndFlrSF: Second floor square feet

LowQualFinSF: Low quality finished square feet (all floors)

GrLivArea: Above grade (ground) living area square feet

BsmtFullBath: Basement full bathrooms

BsmtHalfBath: Basement half bathrooms

FullBath: Full bathrooms above grade

HalfBath: Half baths above grade

Bedroom: Bedrooms above grade (does NOT include basement bedrooms)

Kitchen: Kitchens above grade

KitchenQual: Kitchen quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       	
TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)

Functional: Home functionality (Assume typical unless deductions are warranted)

       Typ	Typical Functionality
       Min1	Minor Deductions 1
       Min2	Minor Deductions 2
       Mod	Moderate Deductions
       Maj1	Major Deductions 1
       Maj2	Major Deductions 2
       Sev	Severely Damaged
       Sal	Salvage only
		
Fireplaces: Number of fireplaces

FireplaceQu: Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace
		
GarageType: Garage location
		
       2Types	More than one type of garage
       Attchd	Attached to home
       Basment	Basement Garage
       BuiltIn	Built-In (Garage part of house - typically has room above garage)
       CarPort	Car Port
       Detchd	Detached from home
       NA	No Garage
		
GarageYrBlt: Year garage was built
		
GarageFinish: Interior finish of the garage

       Fin	Finished
       RFn	Rough Finished	
       Unf	Unfinished
       NA	No Garage
		
GarageCars: Size of garage in car capacity

GarageArea: Size of garage in square feet

GarageQual: Garage quality

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage
		
GarageCond: Garage condition

       Ex	Excellent
       Gd	Good
       TA	Typical/Average
       Fa	Fair
       Po	Poor
       NA	No Garage
		
PavedDrive: Paved driveway

       Y	Paved 
       P	Partial Pavement
       N	Dirt/Gravel
		
WoodDeckSF: Wood deck area in square feet

OpenPorchSF: Open porch area in square feet

EnclosedPorch: Enclosed porch area in square feet

3SsnPorch: Three season porch area in square feet

ScreenPorch: Screen porch area in square feet

PoolArea: Pool area in square feet

PoolQC: Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool
		
Fence: Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence
	
MiscFeature: Miscellaneous feature not covered in other categories
		
       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None
		
MiscVal: $Value of miscellaneous feature

MoSold: Month Sold (MM)

YrSold: Year Sold (YYYY)

SaleType: Type of sale
		
       WD 	Warranty Deed - Conventional
       CWD	Warranty Deed - Cash
       VWD	Warranty Deed - VA Loan
       New	Home just constructed and sold
       COD	Court Officer Deed/Estate
       Con	Contract 15% Down payment regular terms
       ConLw	Contract Low Down payment and low interest
       ConLI	Contract Low Interest
       ConLD	Contract Low Down
       Oth	Other
		
SaleCondition: Condition of sale

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)

In [12]:
# также дополнительно посмотрим на данные
df.groupBy("TotalBath").count().show()

+---------+-----+
|TotalBath|count|
+---------+-----+
|       12|    1|
|        6|  186|
|        3|  129|
|        5|  295|
|        9|    7|
|        4|  456|
|        8|   13|
|        7|  144|
|       10|    1|
|        2|  228|
+---------+-----+



In [13]:
# зададим список категориальных полей для использования в String Indexer

train_string_columns = []

for col, dtype in df.dtypes:
    if dtype == 'string':
        train_string_columns.append(col)

In [14]:
# для каждого категориального поля укажем индексы его значений

indexers = [StringIndexer(inputCol=column, outputCol=column+'_index', handleInvalid='keep').fit(df) for column in train_string_columns]


pipeline = Pipeline(stages=indexers)
train_indexed = pipeline.fit(df).transform(df)

In [15]:
# количество полей
print(len(train_indexed.columns))

148


In [16]:
# оставим только числовые поля

def get_dtype(df,colname):
    return [dtype for name, dtype in df.dtypes if name == colname][0]

num_cols_train = []
for col in train_indexed.columns:
    if get_dtype(train_indexed,col) != 'string':
        num_cols_train.append(str(col))
        
train_indexed = train_indexed.select(num_cols_train)

In [17]:
# количество числовых полей
print(len(train_indexed.columns))

101


In [18]:
# созадим вектор фичей

vectorAssembler = VectorAssembler(inputCols = train_indexed.drop("SalePrice").columns, outputCol = 'features').setHandleInvalid("keep")

train_vector = vectorAssembler.transform(train_indexed)

In [26]:
# посмотрим первые 20 значений поля вектора фичей
train_vector.select("features").show()

+--------------------+
|            features|
+--------------------+
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
|(100,[0,1,2,3,4,5...|
+--------------------+
only showing top 20 rows



In [28]:
# разобьем на тренировочную и тестовую выборки

train, test = train_vector.randomSplit([0.8, 0.2])

In [30]:
# обучим линейную регрессию и посмотрим метрики на тренировочном и тестовом наборе

# обучение модели
lr = LinearRegression(featuresCol = 'features', labelCol='SalePrice', maxIter=15, 
                      regParam=0.9, elasticNetParam=0.01) 
lr_model = lr.fit(train)

# посмотрим 
trainingSummary = lr_model.summary
print("ошибка RMSE, те на сколько в среднем ошибается модель в цене: %f" % trainingSummary.rootMeanSquaredError)
print("R^2 на тренировочных данных: %f" % trainingSummary.r2)

# посмотрим предсказания на тестовом наборе
lr_predictions = lr_model.transform(test)
lr_predictions.select("prediction","SalePrice","features").show(25)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SalePrice", metricName="r2")

print("R^2 на тестовых данных = %g" % lr_evaluator.evaluate(lr_predictions))

ошибка RMSE, те на сколько в среднем ошибается модель в цене: 31261.670891
R^2 на тренировочных данных: 0.847429
+------------------+---------+--------------------+
|        prediction|SalePrice|            features|
+------------------+---------+--------------------+
| 274025.3005080825|   307000|(100,[0,1,2,3,4,5...|
|213025.25834087856|   200000|(100,[0,1,2,3,4,5...|
| 44700.43092457892|   118000|(100,[0,1,2,3,4,5...|
|125340.88539313257|   129500|(100,[0,1,2,3,4,5...|
|130900.88413914232|   144000|(100,[0,1,2,3,4,5...|
|155264.68944228048|   132000|(100,[0,1,2,3,4,5...|
|114571.87970666934|    90000|(100,[0,1,2,3,4,5...|
| 314221.1045505675|   325300|(100,[0,1,2,3,4,5...|
| 273110.5841920122|   306000|(100,[0,1,2,3,4,5...|
|143515.44415441272|   144000|(100,[0,1,2,3,4,5...|
|166306.06716571684|   177000|(100,[0,1,2,3,4,5...|
|108592.90419156814|   130000|(100,[0,1,2,3,4,5...|
| 209950.3431027154|   196500|(100,[0,1,2,3,4,5...|
|120711.18356778903|   124900|(100,[0,1,2,3,4,5...|
|13